In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
# Bước 1: Chuẩn bị dữ liệu
data = pd.read_csv('clean_training_and_development_data.csv')

# Chuyển đổi ngày tháng sang định dạng datetime
data['Training Date'] = pd.to_datetime(data['Training Date'])

# Tách các thông tin từ cột Training Date
data['Year'] = data['Training Date'].dt.year
data['Month'] = data['Training Date'].dt.month
data['Day'] = data['Training Date'].dt.day
data['Weekday'] = data['Training Date'].dt.weekday  # 0=Monday, 6=Sunday

# Xóa cột Training Date vì đã chuyển đổi thông tin cần thiết
data.drop(columns=['Training Date'], inplace=True)

# Chuyển đổi các biến phân loại thành số
label_encoders = {}
for column in ['Training Program Name', 'Training Type', 'Training Outcome', 'Location', 'Trainer']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Tách dữ liệu thành biến đầu vào (X) và biến mục tiêu (y)
X = data.drop(columns=['Training Outcome'])
y = data['Training Outcome']

# Tách dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Bước 2: Xây dựng mô hình học sâu
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Sử dụng sigmoid cho phân loại nhị phân
])

# Biên dịch mô hình
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Bước 3: Huấn luyện mô hình
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Bước 4: Trực quan hóa kết quả
plt.figure(figsize=(12, 5))

# Trực quan hóa độ chính xác
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Độ chính xác (train)')
plt.plot(history.history['val_accuracy'], label='Độ chính xác (validation)')
plt.title('Độ chính xác mô hình')
plt.xlabel('Epochs')
plt.ylabel('Độ chính xác')
plt.legend()

# Trực quan hóa mất mát
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Mất mát (train)')
plt.plot(history.history['val_loss'], label='Mất mát (validation)')
plt.title('Mất mát mô hình')
plt.xlabel('Epochs')
plt.ylabel('Mất mát')
plt.legend()

plt.show()

# Bước 5: Đánh giá mô hình trên tập kiểm tra
y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))

# Lưu mô hình
model.save('training_model.keras')
